## Env Development  
This will contain tests for developing environment.

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import time

from absl import app
from absl import logging
import sys
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Environment/Simplifications')
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Environment/Variations')
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Agent/DDPG/Test1_Frequent_Returns')

import gin
from six.moves import range
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import

import matplotlib
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
#plt.ioff() 

from tf_agents.agents.ddpg import actor_rnn_network
from tf_agents.agents.ddpg import critic_rnn_network
from tf_agents.agents.ddpg import ddpg_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import suite_dm_control
from tf_agents.environments import tf_py_environment
from tf_agents.environments import py_environment
from tf_agents.environments import wrappers
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

import numpy as np
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import policy_saver
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential

from Env import Env
from Env_LS import Env_LS
from FR_Env import FREnv
from Env_no_tests import Env_NT
from Env_Simple import Env_S
from Env_S_C import Env_S_C
from Env_S_A import Env_S_A
from Env_Transfers import Env_T
from Env_Cull_Counter import Env_CC

max_episode_length=1000
num_herds = 2
total_population = 300
weeks_until_testresults = 0.
average_episode_length = 200

In [17]:
#py_env = Env_NT(num_herds = num_herds, total_population = total_population, fix_episode_length=True, average_episode_length = 200)
py_env = Env(num_herds = num_herds, 
             total_population = total_population, 
             fix_episode_length=True, 
             weeks_until_testresults = weeks_until_testresults,
             average_episode_length = average_episode_length)

py_env_t = Env_T(num_herds = num_herds,
                 total_population = total_population, 
                 fix_episode_length=True, 
                 weeks_until_testresults = weeks_until_testresults,
                 average_episode_length = average_episode_length)

py_env_cc = Env_CC(num_herds = num_herds,
                   total_population = total_population, 
                   fix_episode_length=True, 
                   weeks_until_testresults = weeks_until_testresults,
                   average_episode_length = average_episode_length)

First, test environment input and output.

In [3]:
if(num_herds==2):
    zero_step = py_env.reset()
    print('Reset: ', zero_step)
    for i in range (0,10):
        step = py_env.step([0.,0.,0.,0.])
        print('Step: ', step)
else:
    print('Not for more than 2 herds.')

Reset:  TimeStep(
{'discount': 1.0,
 'observation': array([0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'reward': 0.0,
 'step_type': array(0, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.005, 0.005, 0.   , 0.   , 0.005, 0.   , 0.   ], dtype=float32),
 'reward': array(-0.16, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.01, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  ], dtype=float32),
 'reward': array(-0.17333333, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.015, 0.015, 0.   , 0.   , 0.015, 0.   , 0.   ], dtype=float32),
 'reward': array(-0.17333333, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.02, 0.02, 0.  , 0.  , 0.02, 0.  , 0.  ], dtype=float32),
 'reward': array(-0.18666667, 

Define two scripted policies:

In [4]:
action_script1 = [(16, [0,0,0,0]), 
                 (1, [0,0,1,1]),
                 (16, [0,0,0,0]), 
                 (1, [0,0,1,1])] * int(1+max_episode_length)

scr_pol_1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script1)

action_script2 = [(17, [0,0,0,0]), 
                 (1, [0,0,1,1]),
                 (17, [0,0,0,0]), 
                 (1, [0,0,1,1])] * int(1+max_episode_length)

scr_pol_2 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script2)

'''action_script1 = [(5, [0,0]), 
                 (1, [1,1]),
                 (5, [0,0]), 
                 (1, [1,1])] * int(1+max_episode_length)

scr_pol_1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script1)

action_script2 = [(6, [0,0]), 
                 (1, [1,1]),
                 (6, [0,0]), 
                 (1, [1,1])] * int(1+max_episode_length)

scr_pol_2 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script2)'''

'action_script1 = [(5, [0,0]), \n                 (1, [1,1]),\n                 (5, [0,0]), \n                 (1, [1,1])] * int(1+max_episode_length)\n\nscr_pol_1 = scripted_py_policy.ScriptedPyPolicy(\n    time_step_spec=py_env.time_step_spec(),\n    action_spec=py_env.action_spec(),\n    action_script=action_script1)\n\naction_script2 = [(6, [0,0]), \n                 (1, [1,1]),\n                 (6, [0,0]), \n                 (1, [1,1])] * int(1+max_episode_length)\n\nscr_pol_2 = scripted_py_policy.ScriptedPyPolicy(\n    time_step_spec=py_env.time_step_spec(),\n    action_spec=py_env.action_spec(),\n    action_script=action_script2)'

And create a random policy:

In [5]:
random_policy = random_py_policy.RandomPyPolicy(time_step_spec=py_env.time_step_spec(), 
                                                action_spec=py_env.action_spec())

Now write a function that tests an environment with any policy.  
Outputs average return over a set number of episodes and average steps where the agent culled one or more herds.

In [6]:
def test_rnn_env(environment, policy, num_episodes = 50, num_herds = 2):
    if isinstance(environment, py_environment.PyEnvironment):
        total_return = 0.0
        cullsteps = 0 
        if environment.action_spec().shape[0] == num_herds:
            only_culls = True
        else:
            only_culls = False
        for e in range(num_episodes):

            time_step = environment.reset()
            if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
                policy_state = policy.get_initial_state() # remember where in the script we were
            else:
                #print(policy.get_initial_state(batch_size=train_env.batch_size()))
                policy_state = policy.get_initial_state(batch_size=1) # other policies without memory
            episode_return = 0.0
            i=0
            while not time_step.is_last():
                i+=1
                action_step = policy.action(time_step, policy_state)
                if only_culls:
                    for j in range (0, num_herds):
                        if action_step.action[j] >= 0.5:
                            cullsteps += 1
                else:
                    for j in range (num_herds, num_herds*2):
                        if action_step.action[j] >= 0.5:
                            cullsteps += 1
                policy_state = action_step.state
                time_step = environment.step(action_step.action)

                episode_return += time_step.reward

            total_return += episode_return
        avg_return = total_return / num_episodes
        cullsteps /= num_episodes
        return avg_return, cullsteps
    else:
        return None

At this point, use the policies for stress testing.

In [7]:
if(num_herds == 2):
    avg_return, culls = test_rnn_env(py_env, scr_pol_1, num_episodes = 200)
    culls /= 2
    print('average return = {0} cullsteps = {1}'.format(avg_return, culls))

average return = -32.755866742366926 cullsteps = 11.0


In [8]:
if(num_herds == 2):
    avg_return, culls = test_rnn_env(py_env, scr_pol_2, num_episodes = 200)
    culls /= 2
    print('average return = {0} cullsteps = {1}'.format(avg_return, culls))

average return = -34.05886674197391 cullsteps = 11.0


In [9]:
avg_return, culls = test_rnn_env(py_env, random_policy , num_episodes = 100)
culls /= 2
print('average return = {0} cullsteps = {1}'.format(avg_return, culls))

average return = -199.67128115974018 cullsteps = 99.255


Additionally, run a Pseudo-Agent. Currently only works for 2 Herds / Actions with test.

In [10]:
def PseudoAgent(env, num_episodes):
    assert num_episodes >= 0, "Please enter a positive integer for episode number."
    # Outputs
    total_culls = np.zeros((num_herds,), np.int32)
    total_return = 0.
    testresults_time = weeks_until_testresults / average_episode_length
    
    for i in range (0, num_episodes):
        time_step = env.reset()
        episode_return = 0.
        
        while not time_step.is_last():
            cull_all = False
            act = np.zeros((num_herds*2,), np.float32)
            for k in range (0, num_herds):
                act[k] = 0.5
            for l in range (0, num_herds):
                if (np.float32(time_step.observation[(l+1)*3]) >= np.float32(0.1) 
                    and np.float32(time_step.observation[((l+1)*3)-2]) == np.float32(testresults_time)):
                    cull_all = True
                    break
            if cull_all:
                for j in range (num_herds,num_herds*2):
                    act[j] = 1.
                    total_culls[j-num_herds] += 1
            time_step = env.step(act)
            episode_return += time_step.reward
        total_return += episode_return
        
    culls = np.sum(total_culls) / num_episodes
    avg_return = total_return / num_episodes
    return culls, avg_return

In [11]:
def PseudoAgent2(env, num_episodes):
    assert num_episodes >= 0, "Please enter a positive integer for episode number."
    # Outputs
    total_culls = np.zeros((num_herds,), np.int32)
    total_return = 0.
    testresults_time = weeks_until_testresults / average_episode_length
    
    for i in range (0, num_episodes):
        time_step = env.reset()
        episode_return = 0.
        
        while not time_step.is_last():
            act = np.zeros((num_herds*2,), np.float32)
            for k in range (0, num_herds):
                act[k] = 0.5
            for l in range (0, num_herds):
                if (np.float32(time_step.observation[(l+1)*3]) >= np.float32(0.1) 
                    and np.float32(time_step.observation[((l+1)*3)-2]) == np.float32(testresults_time)):
                    act[l+num_herds] = 1.
                    total_culls[l] += 1
            time_step = env.step(act)
            episode_return += time_step.reward
        total_return += episode_return
        
    culls = np.sum(total_culls) / num_episodes
    avg_return = total_return / num_episodes
    return culls, avg_return

In [12]:
c , ar = PseudoAgent(py_env, 500)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)

Average Culls per Episode:  13.236
Average Return:  -30.62954714883212


In [13]:
c , ar = PseudoAgent(py_env_t, 500)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)

Average Culls per Episode:  13.472
Average Return:  -30.255493829206564


In [18]:
c , ar = PseudoAgent(py_env_cc, 500)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)

Average Culls per Episode:  13.204
Average Return:  -30.925520241813736


In [15]:
c, ar = PseudoAgent2(py_env, 500)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)

Average Culls per Episode:  14.052
Average Return:  -40.44474675542582
